In [1]:
import torch
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import ChatTTS
from IPython.display import Audio

from modelscope.hub.check_model import check_model_is_id
from modelscope.hub.snapshot_download import snapshot_download

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
2024-06-01 22:07:59,569 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-06-01 22:07:59,571 - modelscope - INFO - Loading ast index from /Users/chen/.cache/modelscope/ast_indexer
2024-06-01 22:07:59,799 - modelscope - INFO - Loading done! Current index file version is 1.13.2, with md5 86c558a9e8cb0062281e3a14873aae1c and a total number of 972 components indexed


## Load Models

In [2]:
model_id = 'pzc163/chatTTS'
is_id = check_model_is_id(model_id)
model_dir = snapshot_download(model_id)

chat = ChatTTS.Chat()
chat.load_models(source='local', local_path=model_dir)

# Use force_redownload=True if the weights updated.
# chat.load_models(force_redownload=True)

# If you download the weights manually, set source='locals'.
# chat.load_models(source='local', local_path='YOUR LOCAL PATH')

INFO:ChatTTS.core:Load from local: /Users/chen/.cache/modelscope/hub/pzc163/chatTTS
INFO:ChatTTS.core:use cpu
INFO:ChatTTS.core:vocos loaded.
INFO:ChatTTS.core:dvae loaded.
INFO:ChatTTS.core:gpt loaded.
INFO:ChatTTS.core:decoder loaded.
INFO:ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.core:All initialized.


## Inference

### Batch infer

In [ ]:
texts =  ["我觉得像我们这些写程序的人，他，我觉得多多少少可能会对开源有一种情怀在吧我觉得开源是一个很好的形式。现在其实最先进的技术掌握在一些公司的手里的话，就他们并不会轻易的开放给所有的人用。"]     
        
wavs = chat.infer(texts)

INFO:ChatTTS.core:All initialized.
 31%|███       | 630/2048 [01:04<02:50,  8.30it/s]

In [14]:
Audio(wavs[0], rate=24_000, autoplay=True)

### Custom params

In [15]:
params_infer_code = {'prompt':'[speed_5]', 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}

wav = chat.infer('四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
 19%|█▉        | 399/2048 [00:54<03:45,  7.32it/s]


In [17]:
Audio(wav[0], rate=24_000, autoplay=True)

### fix random speaker

In [ ]:
rand_spk = chat.sample_random_speaker()
params_infer_code = {'spk_emb' : rand_spk, }

wav = chat.infer('四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

### Two stage control

In [20]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
chat.infer(text, refine_text_only=True)

INFO:ChatTTS.core:All initialized.
 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
INFO:NeMo-text-processing:Creating ClassifyFst grammars.
 22%|██▏       | 84/384 [00:14<00:51,  5.81it/s]


['so we found [uv_break] being competitive and collaborative [uv_break] was a huge way of staying motivated towards [uv_break] our goals, [uv_break] so [uv_break] one person to call when you fall off, [uv_break] one person who gets you back on [uv_break] then one person to actually do the activity with.']

In [19]:
text = 'so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.'
wav = chat.infer(text, skip_refine_text=True)

INFO:ChatTTS.core:All initialized.


UnboundLocalError: cannot access local variable 'Normalizer' where it is not associated with a value

## LLM Call

In [ ]:
from ChatTTS.experimental.llm import llm_api

API_KEY = ''
client = llm_api(api_key=API_KEY,
        base_url="https://api.deepseek.com",
        model="deepseek-chat")

In [ ]:
user_question = '四川有哪些好吃的美食呢?'
text = client.call(user_question, prompt_version = 'deepseek')
print(text)
text = client.call(text, prompt_version = 'deepseek_TN')
print(text)

In [ ]:
params_infer_code = {'spk_emb' : rand_spk, 'temperature':.3}

wav = chat.infer(text, params_infer_code=params_infer_code)